In [1]:
import torch
import numpy as np
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scarches as sca
from sklearn.metrics import classification_report

from scarches.dataset.trvae.data_handling import remove_sparsity
from lataq.models import EMBEDCVAE
from lataq.exp_dict import EXPERIMENT_INFO
import time
sc.settings.set_figure_params(dpi=200, frameon=False)
sc.set_figure_params(dpi=500)
plt.rcParams['figure.figsize'] = (5, 5)
torch.set_printoptions(precision=3, sci_mode=False, edgeitems=7)

%load_ext autoreload
%autoreload 2

In [2]:
adata = sc.read(
    '../data/hlca_counts_commonvars.h5ad'
)
#adata = adata[:, adata.var.highly_variable].copy()
#adata.X

In [3]:
condition_key = 'sample'
cell_type_key = ['ann_finest_level']

In [4]:
vae_epochs = 50
scanvi_epochs = 20

early_stopping_kwargs = {
    "early_stopping_metric": "elbo",
    "save_best_state_metric": "elbo",
    "patience": 10,
    "threshold": 0,
    "reduce_lr_on_plateau": True,
    "lr_patience": 8,
    "lr_factor": 0.1,
}
early_stopping_kwargs_scanvi = {
    "early_stopping_metric": "accuracy",
    "save_best_state_metric": "accuracy",
    "on": "full_dataset",
    "patience": 10,
    "threshold": 0.001,
    "reduce_lr_on_plateau": True,
    "lr_patience": 8,
    "lr_factor": 0.1,
}

In [5]:
sca.dataset.setup_anndata(
    adata, batch_key=condition_key, labels_key=cell_type_key[0]
)

vae = sca.models.SCANVI(
    adata,
    None,
    n_layers=2,
    n_latent = 30, # to allow for capturing more heterogeneity
    encode_covariates=True,
    deeply_inject_covariates=False,
    use_layer_norm="both",
    use_batch_norm="none",
    gene_likelihood="nb", # because we have UMI data
)

INFO     Using batches from adata.obs["sample"]                                              
INFO     Using labels from adata.obs["ann_finest_level"]                                     
INFO     Using data from adata.X                                                             
INFO     Computing library size prior per batch                                              
INFO     Successfully registered anndata object containing 584884 cells, 1897 vars, 166      
         batches, 58 labels, and 0 proteins. Also registered 0 extra categorical covariates  
         and 0 extra continuous covariates.                                                  
INFO     Please do not further modify adata until model is trained.                          


/home/icb/carlo.dedonno/anaconda3/envs/lataq_cuda/lib/python3.8/site-packages/scvi/model/base/_base_model.py:149: UserWarning: Make sure the registered X field in anndata contains unnormalized count data.
  warnings.warn(


In [6]:
vae.train(
    max_epochs=50
)

INFO     Training for 50 epochs.                                                             


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 50/50: 100%|██████████| 50/50 [1:15:52<00:00, 91.05s/it, loss=714, v_num=1]


In [7]:
vae.save(f"scanvi_hlca_sample", overwrite=True)

In [8]:
adata_latent = sc.AnnData(X=vae.get_latent_representation(), obs=adata.obs)

/home/icb/carlo.dedonno/anaconda3/envs/lataq_cuda/lib/python3.8/site-packages/scvi/model/base/_base_model.py:149: UserWarning: Make sure the registered X field in anndata contains unnormalized count data.
  warnings.warn(


In [9]:
adata_latent.write('../figure_notebooks/hlca_core_scanvi_sample.h5ad')